# Proposal: Fitting with dataset(s)

## Preparation

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
torch.manual_seed(0);

In [3]:
import numpy as np
from sklearn.datasets import make_classification

In [4]:
X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)

In [5]:
X.shape, y.shape, y.mean()

((1000, 20), (1000,), 0.5)

In [6]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin
        self.dropout = dropout

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X), dim=-1)
        return X

## The olde way

In [7]:
from skorch.net import NeuralNetClassifier

In [8]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    # use_cuda=True,  # uncomment this to train with CUDA
)

In [9]:
net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6868       0.6000        0.6740  1.1114
      2        0.6706       0.6400        0.6617  0.1148
      3        0.6637       0.6650        0.6504  0.1116
      4        0.6548       0.7000        0.6418  0.1306
      5        0.6340       0.7100        0.6272  0.1369
      6        0.6219       0.7150        0.6124  0.1201
      7        0.6058       0.7100        0.5980  0.1252
      8        0.5964       0.7200        0.5875  0.1103
      9        0.5901       0.7100        0.5760  0.1254
     10        0.5716       0.7250        0.5651  0.1206
     11        0.5633       0.7250        0.5580  0.1261
     12        0.5652       0.7300        0.5529  0.1006
     13        0.5462       0.7350        0.5426  0.1419
     14        0.5407       0.7300        0.5407  0.1165
     15        0.5360       0.7300        0.5373  0.1350
     16        0.5517       0.7

<class 'skorch.net.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
)

## with dataset, without validation

In [10]:
import torch.utils.data

In [11]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
        assert len(X) == len(y)
        
    def __len__(self):
        return len(self.X)
        
    def __getitem__(self, i):
        return self.X[i], self.y[i]

In [12]:
ds = MyDataset(X, y)

In [13]:
net.fit(ds, None)

Re-initializing module!
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7114  0.0945
      2        0.6894  0.1207
      3        0.6826  0.1092
      4        0.6770  0.1128
      5        0.6586  0.0748
      6        0.6494  0.0955
      7        0.6265  0.0894
      8        0.6280  0.1124
      9        0.6195  0.0907
     10        0.5944  0.1081
     11        0.5889  0.1124
     12        0.5829  0.1166
     13        0.5605  0.0978
     14        0.5620  0.0942
     15        0.5678  0.1012
     16        0.5468  0.1069
     17        0.5525  0.1229
     18        0.5421  0.0604
     19        0.5472  0.0981
     20        0.5345  0.0832


<class 'skorch.net.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
)

## with dataset and validation

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [16]:
ds_train, ds_valid = MyDataset(X_train, y_train), MyDataset(X_valid, y_valid)

to get validation, pass X as a tuple of datasets

### this does not work

In [17]:
try:
    net.fit((ds_train, ds_valid), (None, None))
except Exception as e:
    print(e)

Re-initializing module!
__call__() missing 1 required positional argument: 'y_true'


### this works

but once we re-use data from batches for scoring, the way above should also work!

In [18]:
net.fit((ds_train, ds_valid), (y_train, y_valid))

Re-initializing module!
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7179       0.5280        0.6938  0.0992
      2        0.6971       0.5440        0.6893  0.0825
      3        0.6872       0.5720        0.6842  0.0832
      4        0.6827       0.5920        0.6787  0.0873
      5        0.6746       0.6040        0.6726  0.0860
      6        0.6732       0.6120        0.6679  0.0964
      7        0.6707       0.6360        0.6624  0.0947
      8        0.6644       0.6480        0.6570  0.0710
      9        0.6637       0.6560        0.6508  0.0862
     10        0.6508       0.6480        0.6435  0.0841
     11        0.6503       0.6560        0.6366  0.0831
     12        0.6438       0.6800        0.6302  0.0652
     13        0.6432       0.6680        0.6238  0.0840
     14        0.6282       0.6960        0.6134  0.0707
     15        0.6158       0.7040        0.6024  0.0800
     16

<class 'skorch.net.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
)

## Predictions

In [19]:
_ = net.predict(ds)
_.shape

(1000,)

In [20]:
_ = net.predict_proba(ds)
_.shape

(1000, 2)

## with custom callback

In [21]:
from skorch.callbacks import EpochScoring

In [22]:
auc = EpochScoring(scoring='roc_auc', lower_is_better=False)

In [23]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    callbacks=[auc],
)

In [24]:
net.fit((ds_train, ds_valid), (y_train, y_valid))

  epoch    roc_auc    train_loss    valid_acc    valid_loss     dur
-------  ---------  ------------  -----------  ------------  ------
      1     0.5958        0.6962       0.5560        0.6836  0.0675
      2     0.6372        0.6894       0.5560        0.6777  0.0726
      3     0.6646        0.6753       0.5960        0.6698  0.0815
      4     0.6813        0.6638       0.6080        0.6619  0.0811
      5     0.6987        0.6568       0.6360        0.6549  0.0960
      6     0.7130        0.6533       0.6440        0.6460  0.0800
      7     0.7317        0.6522       0.6720        0.6365  0.0745
      8     0.7443        0.6419       0.6840        0.6281  0.0833
      9     0.7622        0.6287       0.6920        0.6167  0.0897
     10     0.7732        0.6246       0.7000        0.6068  0.1225
     11     0.7839        0.6173       0.7200        0.5972  0.0963
     12     0.7861        0.5985       0.7240        0.5882  0.0853
     13     0.7979        0.5972       0.7360   

<class 'skorch.net.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
)